In [1]:
from RESTInteractions import CRABRest
from ServerUtilities import encodeRequest


In [2]:
host='cmsweb-test2.cern.ch'; dbInstance='dev'
task='230127_111257:belforte_crab_20230127_121253' # test2

#host='cmsweb.cern.ch'; dbInstance='prod'

#host='cmsweb-testbed.cern.ch'; dbInstance='preprod'
#task = '230131_093942:belforte_crab_20230131_103931' # preprod



In [3]:
import logging
import sys
import os
from pprint import pprint
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('Logger')

proxy = os.environ['X509_USER_PROXY']
print("Proxy: %s" % proxy)

Proxy: /tmp/x509up_u8516


In [4]:
server = CRABRest(host, proxy, proxy, userAgent='CRABtestSB',verbose=False)
# this means that we will use URL's like  https://cmsweb-test1.cern.ch/crabserver


In [5]:
server.setDbInstance(dbInstance)
# this means that we will use URL's like https://cmsweb-test1.cern.ch/crabserver/dev

In [6]:
crabServer=server

In [7]:
user = task.split(':')[1].split('_')[0]
print(user)

belforte


In [8]:
# this API is generic
# a PUT to it inserts a new record but
# a POST or GET performs action specified via subresource field
api = 'filetransfers'
# means that we will use URL's like https://cmsweb-test1.cern.ch/crabserver/dev/filetransfer

In [9]:
# this API is generic
# a PUT to it inserts a new record but
# a POST or GET performs action specified via subresource field
api = 'fileusertransfers'
# means that we will use URL's like https://cmsweb-test1.cern.ch/crabserver/dev/filetransfer
# GET supports also querying w/o a subresource
data = {'subresource':'getTransferStatus', 'taskname':task, 'username':user}
data = encodeRequest(data) 
# this means thar URL will be
url = 'https://' + host + ':8443/crabserver/' + dbInstance + '/' + api
finalUrl=url+'?'+data
print(finalUrl)

https://cmsweb-test2.cern.ch:8443/crabserver/dev/fileusertransfers?subresource=getTransferStatus&taskname=230127_111257%3Abelforte_crab_20230127_121253&username=belforte


In [10]:
# now query it
res=crabServer.get(api=api, data=data)
#pprint(res)

In [11]:
result=res[0]['result']
numFiles=len(result)
print("there are %d fileIds" %numFiles)
fileIds=[]
for i in range(numFiles):
    fileIds.append(result[i][0])
pprint(fileIds)
#    print(result[i][0])

there are 2 fileIds
['3be122bd56444c3f99146f9242042eed29ff521358d570b0eb741f9a',
 '0894bf139082f54b13fc811f1861a416135fa5493a5aa3e88f05e133']


In [24]:
# get full info for first file
data={'subresource': 'getById', 'id':fileIds[0]}
data=encodeRequest(data)
finalUrl=url+'?'+data
print(finalUrl)
res=crabServer.get(api=api, data=data)
result=res[0]
#pprint(res)
keys=result['desc']['columns']
values=result['result'][0]
#print(len(keys))
#print(len(values))
for i in range(len(keys)):
    print(keys[i], values[i])

https://cmsweb-test2.cern.ch:8443/crabserver/dev/fileusertransfers?subresource=getById&id=3be122bd56444c3f99146f9242042eed29ff521358d570b0eb741f9a
tm_id 3be122bd56444c3f99146f9242042eed29ff521358d570b0eb741f9a
tm_username belforte
tm_taskname 230127_111257:belforte_crab_20230127_121253
tm_destination T2_CH_CERN
tm_destination_lfn /store/user/belforte/GenericTTbar/crab_20230127_121253/230127_111257/0000/kk_1.root
tm_source T2_CH_CERN
tm_source_lfn /store/temp/user/belforte.be1f4dc5be8664cbd145bf008f5399adf42b086f/GenericTTbar/crab_20230127_121253/230127_111257/0000/kk_1.root
tm_filesize 561301
tm_publish 1
tm_jobid 1
tm_job_retry_count 0
tm_type output
tm_aso_worker schedd
tm_transfer_retry_count 0
tm_transfer_max_retry_count 2
tm_publication_retry_count 1
tm_publication_max_retry_count 2
tm_rest_host None
tm_rest_uri None
tm_transfer_state 3
tm_publication_state 3
tm_transfer_failure_reason None
tm_publication_failure_reason None
tm_fts_id None
tm_fts_instance None
tm_last_update 16753

In [23]:
# try to change one value.
# This uses POST with  the other API 'filetransfers'
# and subresource == 'updateDoc'
# like is being done in PostJob
# this is mean to be used in RUCIO_Transfers.py to
# communicate block info to publisher


data = {'subresource':'updateRucioInfo'}
data['list_of_ids'] = [fileIds[0]]
data['asoworker'] = 'schedd'
data['list_of_dbs_blockname'] = ['/PrimDsTRETRE/ProcDs/TIER#ablock']
data['list_of_block_complete'] = ['TRE']
pprint(data)

res=crabServer.post(api='filetransfers', data=encodeRequest(data))

pprint(res)

{'asoworker': 'schedd',
 'list_of_block_complete': ['TRE'],
 'list_of_dbs_blockname': ['/PrimDsTRETRE/ProcDs/TIER#ablock'],
 'list_of_ids': ['3be122bd56444c3f99146f9242042eed29ff521358d570b0eb741f9a'],
 'subresource': 'updateRucioInfo'}
({'result': ['']}, 200, 'OK')


In [ ]:
# make sure to che